<a href="https://colab.research.google.com/github/ytg000629/Dacon_Dobae/blob/main/modeling/pytorch_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler


import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
from torchvision import transforms

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
device

device(type='cuda')

In [4]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
all_img_list = glob.glob('/content/drive/MyDrive/dacon/data/train/*/*')

In [8]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])

In [9]:
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['label'])

In [10]:
df.head()

,img_path,label
0,/content/drive/MyDrive/dacon/data/train/가구수...,0
1,/content/drive/MyDrive/dacon/data/train/가구수...,0
2,/content/drive/MyDrive/dacon/data/train/가구수...,0
3,/content/drive/MyDrive/dacon/data/train/가구수...,0
4,/content/drive/MyDrive/dacon/data/train/가구수...,0


In [ ]:
df['label'].nunique

In [ ]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

In [ ]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

In [ ]:
val

,img_path,label
133,/content/drive/MyDrive/dacon/data/train/걸레ᄇ...,1
3151,/content/drive/MyDrive/dacon/data/train/훼손/...,18
12,/content/drive/MyDrive/dacon/data/train/걸레ᄇ...,1
1795,/content/drive/MyDrive/dacon/data/train/오염/...,10
3353,/content/drive/MyDrive/dacon/data/train/훼손/...,18
...,...,...
1359,/content/drive/MyDrive/dacon/data/train/터짐/...,15
1595,/content/drive/MyDrive/dacon/data/train/오염/...,10
2318,/content/drive/MyDrive/dacon/data/train/훼손/...,18
631,/content/drive/MyDrive/dacon/data/train/꼬임/...,3


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)


        data_with_augmentation = ImageDataGenerator(width_shift_range = 0.1,
                                            height_shift_range = 0.1,
																						horizontal_flip = True,
																						vertical_flip = True,
                                            fill_mode = 'nearest',
                                            #shear_range = 0.1,
                                            rescale = 1/255.0) # train data 증강 방식

In [ ]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.OneOf([A.HorizontalFlip(p=1),A.RandomRotate90(p=1),A.VerticalFlip(p=1)], p=1),
                            A.OneOf([A.MotionBlur(p=1),A.OpticalDistortion(p=1),A.GaussNoise(p=1)], p=1),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True) #efficientnet_b0
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x): ##FFN
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 31.8MB/s]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.40680] Val Loss : [0.92112] Val Weighted F1 Score : [0.69029]


  0%|          | 0/76 [00:00<?, ?it/s]

...

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.04753] Val Loss : [1.11256] Val Weighted F1 Score : [0.76260]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.01719] Val Loss : [1.12584] Val Weighted F1 Score : [0.75670]


In [ ]:
test_list = glob.glob('/content/drive/MyDrive/dacon/data/test/*')

In [ ]:
test = pd.DataFrame(test_list, columns = ['img_path']).sort_values('img_path')

In [ ]:
test

,img_path
25,/content/drive/MyDrive/dacon/data/test/000.png
701,/content/drive/MyDrive/dacon/data/test/001.png
464,/content/drive/MyDrive/dacon/data/test/002.png
73,/content/drive/MyDrive/dacon/data/test/003.png
195,/content/drive/MyDrive/dacon/data/test/004.png
...,...
772,/content/drive/MyDrive/dacon/data/test/787.png
778,/content/drive/MyDrive/dacon/data/test/788.png
652,/content/drive/MyDrive/dacon/data/test/789.png
711,/content/drive/MyDrive/dacon/data/test/790.png


In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
def label_decoder(label):
    new_label =[
        '가구수정' if l==0 else
        '걸레받이수정' if l==1 else
        '곰팡이' if l==2 else
        '꼬임' if l==3 else
        '녹오염' if l==4 else
        '들뜸' if l==5 else
        '면불량' if l==6 else
        '몰딩수정' if l==7 else
        '반점' if l==8 else
        '석고수정' if l==9 else
        '오염' if l==10 else
        '오타공' if l==11 else
        '울음' if l==12 else
        '이음부불량' if l==13 else
        '창틀,문틀수정' if l==14 else
        '터짐' if l==15 else
        '틈새과다' if l==16 else
        '피스' if l==17 else
        '훼손' if l==18 else
        'NaN' for l in label
    ]
    return np.array(new_label)


In [ ]:
preds = label_decoder(preds)

In [ ]:
preds[:5]

array(['훼손', '오염', '훼손', '몰딩수정', '오염'], dtype='<U7')

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/dacon/data/sample_submission.csv')

In [ ]:
submit['label'] = preds

In [ ]:
submit.head()

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염


In [ ]:
submit.to_csv('/content/drive/MyDrive/dacon/submit/0518_12.csv', index=False, encoding='utf-8-sig')

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# import package
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

# model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch import optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

# dataset and transformation
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import time
import copy

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [ ]:
all_img_list = glob.glob('/content/drive/MyDrive/dacon/data/train/*/*')

In [ ]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])

In [10]:
# specift the data path
train_data_dir = "/content/drive/MyDrive/dacon/data/data_split/train"
val_data_dir = '/content/drive/MyDrive/dacon/data/data_split/val'
test_data_dir = '/content/drive/MyDrive/dacon/data/test'

# if not exists the path, make the directory
#if not os.path.exists(path2data):
    #os.mkdir(path2data)

# load dataset
train_ds = datasets.STL10(train_data_dir, split='train', download=True, transform=transforms.ToTensor())
val_ds = datasets.STL10(val_data_dir, split='test', download=True, transform=transforms.ToTensor())

print(len(train_ds))
print(len(val_ds))

100%|██████████| 2640397119/2640397119 [01:54<00:00, 23074375.34it/s]


Extracting /content/drive/MyDrive/dacon/data/data_split/train/stl10_binary.tar.gz to /content/drive/MyDrive/dacon/data/data_split/train


100%|██████████| 2640397119/2640397119 [02:06<00:00, 20860028.77it/s]


Extracting /content/drive/MyDrive/dacon/data/data_split/val/stl10_binary.tar.gz to /content/drive/MyDrive/dacon/data/data_split/val
5000
8000


In [ ]:
# create dataloader
train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=8, shuffle=True)

#batch_size 크게 설정하면 메모리 터짐

In [ ]:
# display sample images
def show(img, y=None, color=True): 
    npimg = img.numpy()
    npimg_tr = np.transpose(npimg, (1, 2, 0))
    plt.imshow(npimg_tr)

    if y is not None:
        plt.title('labels:' + str(y))

np.random.seed(0)
torch.manual_seed(0)

grid_size = 4
rnd_ind = np.random.randint(0, len(train_ds), grid_size)

x_grid = [train_ds[i][0] for i in rnd_ind]
y_grid = [train_ds[i][1] for i in rnd_ind]

plt.figure(figsize=(10,10))
x_grid = utils.make_grid(x_grid, nrow=4, padding=2)
show(x_grid, y_grid)